<a href="https://colab.research.google.com/github/gorae17/Colab_test/blob/master/0414.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 삭제
from google.colab import drive
drive.mount('/gdrive')
!ls /gdrive/My\ Drive/Colab\ Notebooks/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
 캐글스터디	      '0412.ipynb의 사본'       AClassification.train.txt
 04122.ipynb	      '0412.ipynb의 사본 (1)'   models
'04122.ipynb의 사본'   0413.ipynb	        MultiLabelClassification.ipynb
 0412.ipynb	       0414.ipynb	        Untitled


In [0]:
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score
#from sklearn.metrics import accuracy_score

Ep = sys.float_info.epsilon

In [0]:
def final_predict(x):
  re = [0 for i in range(0,len(x))]
  arg = np.array(x).argsort()
  re[arg[-1]] = 1
  for idx in arg[-3:-1]:
    if x[idx] < 0.6:
      re[idx] = 0
    else:
      re[idx] = 1
  return re

In [0]:
data = open('/gdrive/My Drive/Colab Notebooks/AClassification.train.txt','r').readlines()
data = [line.strip().split('\t') for line in data]
df = pd.DataFrame.from_records(data, columns=['id','labels','features'])
df['features'] = df['features'].apply(lambda x: " ".join([tok.split('/')[0] for tok in x.split(' ')[:300]]))
df['labels'] = df['labels'].apply(lambda x: [int(y) for y in x.split()])
categories = list(range(0, 20))
for i in categories:
  df[i] = df['labels'].apply(lambda x: 1 if i in x else 0)

In [0]:
train, test = train_test_split(df, random_state=17, test_size=0.3, shuffle=True)
x_train = train.features
x_test = test.features
trainedmodel = {}

vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)
x_train_v = vectorizer.transform(x_train)
x_test_v = vectorizer.transform(x_test)                             

In [11]:
from lightgbm import LGBMRegressor

for category in categories:

  print('... Processing {}'.format(category))
  
  lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=4, 
                           subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)

  lgbm_reg.fit(x_train_v, train[category])
  trainedmodel['LGBM{}'.format(category)] = lgbm_reg
  
  test['p{}'.format(category)] = np.array(lgbm_reg.predict(x_test_v))

col = ['id','labels','features'] + ['p{}'.format(x) for x in range(0,20)]
y_test = test.drop(columns=col)
col = ['id','labels','features'] + list(range(0,20))
y_pred = test.drop(columns=col)
#y_pred = np.array([res for res in y_pred.apply(lambda x: final_predict(x), axis=1)])

print('f0.5 score {}'.format(fbeta_score(np.array(y_test),np.array(y_pred),0.5,average='micro')))

... Processing 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


... Processing 1
... Processing 2
... Processing 3
... Processing 4
... Processing 5
... Processing 6
... Processing 7
... Processing 8
... Processing 9
... Processing 10
... Processing 11
... Processing 12
... Processing 13
... Processing 14
... Processing 15
... Processing 16
... Processing 17
... Processing 18
... Processing 19


ValueError: ignored

In [22]:

for category in categories:
  test['p{}'.format(category)] = [0 if abs(1-x) > abs(x) else 1 for x in np.array(trainedmodel['LGBM{}'.format(category)].predict(x_test_v))]

col = ['id','labels','features'] + list(range(0,20))
y_pred = test.drop(columns=col)
#y_pred = np.array([res for res in y_pred.apply(lambda x: final_predict(x), axis=1)])

print('f0.5 score {}'.format(fbeta_score(np.array(y_test),np.array(y_pred),0.5,average='micro')))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


f0.5 score 0.8681464873639997


In [20]:
y_pred.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19
242364,0.002795,0.007837,0.004527,0.000752,0.002930,0.580224,-0.011297,0.005742,0.017047,0.000142,0.029697,0.004576,0.016042,0.002455,-0.040028,0.034978,-0.000074,-0.001914,0.201407,0.000957
70377,0.009881,0.014179,0.004539,0.002853,0.000919,0.066988,0.001322,0.011064,0.016006,0.001137,0.011614,0.877641,0.024530,0.002455,0.000253,0.004064,0.018970,-0.000211,0.000038,0.000957
124907,0.000102,0.001700,-0.000049,0.000752,0.001776,-0.000216,0.414224,-0.000091,0.001161,-0.000058,0.000459,0.000085,-0.003584,0.085750,0.137093,-0.004562,-0.000074,-0.000211,0.475436,-0.000111
280141,0.009881,0.014179,0.016790,0.002853,0.003887,0.764771,0.001322,0.011438,0.026647,0.000397,0.019147,0.014426,0.022165,0.002455,0.000253,0.003760,0.000132,-0.000211,0.000038,0.000957
126377,0.003930,0.007385,0.006211,0.002853,0.002500,0.023087,1.103708,0.006302,0.010122,-0.003037,0.008553,0.006302,0.207928,0.003230,0.089546,-0.002521,0.000132,-0.000211,1.030747,0.003657


In [11]:
from lightgbm import LGBMRegressor

for category in categories:

  print('... Processing {}'.format(category))
  
  lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=4, 
                           subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)

  lgbm_reg.fit(x_train_v, train[category])
  trainedmodel['LGBM{}'.format(category)] = lgbm_reg
  
  test['p{}'.format(category)] = np.array(lgbm_reg.predict(x_test_v))

col = ['id','labels','features'] + ['p{}'.format(x) for x in range(0,20)]
y_test = test.drop(columns=col)
col = ['id','labels','features'] + list(range(0,20))
y_pred = test.drop(columns=col)
y_pred = np.array([res for res in y_pred.apply(lambda x: final_predict(x), axis=1)])

print('f0.5 score {}'.format(fbeta_score(np.array(y_test),np.array(y_pred),0.5,average='micro')))

... Processing 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


... Processing 1
... Processing 2
... Processing 3
... Processing 4
... Processing 5
... Processing 6
... Processing 7
... Processing 8
... Processing 9
... Processing 10
... Processing 11
... Processing 12
... Processing 13
... Processing 14
... Processing 15
... Processing 16
... Processing 17
... Processing 18
... Processing 19
f0.5 score 0.8626659211861424


# RNN with Keras

In [9]:
import tensorflow as tf
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
K.set_session(tf.Session(config=config))

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
# K.set_session(tf.Session(config=config))

InternalError: ignored

In [0]:
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score
#from sklearn.metrics import accuracy_score

Ep = sys.float_info.epsilon

In [0]:
data = open('/gdrive/My Drive/Colab Notebooks/AClassification.train.txt','r').readlines()
data = [line.strip().split('\t') for line in data]
df = pd.DataFrame.from_records(data, columns=['id','labels','features'])
df['features'] = df['features'].apply(lambda x: " ".join([tok.split('/')[0] for tok in x.split(' ')[:300]]))
df['labels'] = df['labels'].apply(lambda x: [int(y) for y in x.split()])
categories = list(range(0, 20))
for i in categories:
  df[i] = df['labels'].apply(lambda x: 1 if i in x else 0)

In [0]:
train, test = train_test_split(df, random_state=17, test_size=0.3, shuffle=True)
x_train = train.features
x_test = test.features

from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train) #5572개의 행을 가진 X의 각 행에 토큰화를 수행
train_sequences = tokenizer.texts_to_sequences(x_train) #단어를 숫자값, 인덱스로 변환하여 저장
test_sequences = tokenizer.texts_to_sequences(x_test) #단어를 숫자값, 인덱스로 변환하여 저장

from keras.layers import SimpleRNN, Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
max_features = 10000
# feature로 사용할 단어의 수.
maxlen = 300
# e-mail에서 볼 최대 단어의 수는 500개

train_data = pad_sequences(train_sequences, maxlen=maxlen)
test_data = pad_sequences(test_sequences, maxlen=maxlen)


In [8]:
for category in categories:
  print('... Processing {}'.format(category))

  model = Sequential()
  model.add(Embedding(max_features, 32))
  model.add(SimpleRNN(32))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
  history = model.fit(train_data, train[category], epochs=10, batch_size=32, validation_split=0.2)
  print("\n 테스트 정확도: %.4f" % (model.evaluate(test_data, test[category])[1]))
  test['p{}'.format(category)] = model.predict(x_test)
  
col = ['id','labels','features'] + ['p{}'.format(x) for x in range(0,20)]
y_test = test.drop(columns=col)
col = ['id','labels','features'] + list(range(0,20))
y_pred = test.drop(columns=col)
#y_pred = np.array([res for res in y_pred.apply(lambda x: final_predict(x), axis=1)])

print('f0.5 score {}'.format(fbeta_score(np.array(y_test),np.array(y_pred),0.5,average='micro')))


... Processing 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 170340 samples, validate on 42586 samples
Epoch 1/10


InternalError: ignored